<a href="https://colab.research.google.com/github/mariachurches/AprendizajeSupervisado/blob/main/Enunciado_C1_P2/Actividad_C1_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convocatoria 1 - Proyecto 2

### Ejercicio 1 (4 puntos)
Utilizar el conjunto de datos "dataset_1.npy" para resolver el ejercicio. Tener en cuenta que la última columna corresponde a la clase.

In [58]:
#Importamos las librerias necesarias y cargamos
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.covariance import EllipticEnvelope

data = np.load("/content/dataset_1.npy")
X_data = data[:, : -1]
y_data = data[:, -1]

#### 1.a) Imputación de valores ausentes (2 puntos)

- En aquellas instancias (filas) que contengan UN ÚNICO VALOR NaN en alguno de sus atributos (columnas), se imputará dicho valor.
 * Si el atributo corresponde a una variable discreta, se imputará el valor utilizando la moda de dicho atributo. 
 * Si el atributo corresponde a una variable continua, se imputará el valor utilizando la media de dicho atributo. 
- Aquellas instancias (filas) que contengan MÁS DE UN VALOR NaN en sus atributos, deberán ser eliminadas por completo. 

Los outputs deberán ser: 
- Una matriz "X" de dimensiones M x N, donde M será el número de instancias y N, el de atributos.
- Un vector "y" de dimensiones M. 

In [59]:
print("Antes de la imputación de valores ausentes había que " +  str(np.isnan(data).any(axis=1).sum()) + " filas con valores ausentes")

#Primero borramos aquellas instancias que tienen más de un elemento nan
X_data = X_data[~np.isnan(X_data).all(axis=1)]

#Después borramos los NaN restantes como nos indica el enunciado
data_discreta = X_data[:,:4]
data_continua = X_data[:,5:]

imp_media = SimpleImputer(strategy='mean') # Utilizamos la estrategia de la media
imp_moda = SimpleImputer(strategy='most_frequent') # Utilizamos la estrategia de la media

data_discreta_fit = imp_media.fit_transform(data_discreta)
data_continua_fit = imp_moda.fit_transform(data_continua)

X_data[:,:4] = data_discreta_fit
X_data[:,5:] = data_continua_fit

print("Después de la imputación de valores ausentes hay que " +  str(np.isnan(X_data).any(axis=1).sum()) + " filas con valores ausentes")

Antes de la imputación de valores ausentes había que 8 filas con valores ausentes
Después de la imputación de valores ausentes hay que 0 filas con valores ausentes


#### 1.b) Eliminación de valores outliers extremos (2 puntos)
- Aplicar el método de Elliptic Envelope fijando una semilla en 42
- Utilizar el diagrama de caja y bigotes para establecer los umbrales de decisión
- Plotear el diagrama (box and whiskers) obtenido a partir de las puntuaciones calculadas con el método Elliptic Envelope
- Eliminar las instancias que contengan valores outliers extremos

Los outputs deberán ser: 
- Una matriz "X" de dimensiones M' x N, donde M' será el nuevo número de instancias y N, el de atributos.
- Un vector "y" de dimensiones M'. 

In [63]:
# Limpieza de datos: detección de outliers.
outlier_method = EllipticEnvelope().fit(X_data)

# Obtenemos unos valores de puntuación a nivel de instancia para determinar después las que corresponden con valores atípicos
scores_pred = outlier_method.decision_function(X_data)
print(scores_pred)
print(np.shape(scores_pred))

[ 9.99277440e+01  1.02970467e+02  1.01340983e+02  1.07771876e+02
  1.10089596e+02  1.05864943e+02  9.78430705e+01  1.01893182e+02
  1.07431830e+02  1.04605885e+02  1.09115399e+02  1.02179911e+02
  9.42397679e+01  1.06690465e+02  1.06097584e+02  1.10310657e+02
  1.09562995e+02  1.08574230e+02  1.04888271e+02  1.03394557e+02
  1.08163730e+02  1.02496693e+02  1.04249417e+02  1.07769866e+02
  1.05962777e+02  1.08922394e+02  1.04091721e+02  1.03822200e+02
  1.05311964e+02  1.07411965e+02  1.06276777e+02  1.02560908e+02
  1.07568945e+02  1.03237009e+02  1.04449703e+02  1.09434490e+02
  9.96071679e+01  1.07989426e+02  1.06411553e+02  1.03741098e+02
  1.04059236e+02  1.07779048e+02  9.78660800e+01  1.04084013e+02
  1.02223939e+02  9.96256096e+01  1.05891099e+02  1.04293999e+02
  1.01750682e+02  1.09930382e+02  1.06154778e+02  1.08733517e+02
  9.54665374e+01  1.03754431e+02  1.08963845e+02  1.02805636e+02
  1.10396505e+02  1.02307776e+02  1.04433157e+02  1.03603114e+02
  1.09281413e+02  1.07347

/usr/local/lib/python3.7/dist-packages/sklearn/covariance/_robust_covariance.py:189: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-36.369435824887027 > -36.479965227341815). You may want to try with a higher value of support_fraction (current value: 0.509).
  RuntimeWarning,


### Ejercicio 2 (6 puntos)

Crear un programa, con comentarios que aclaren el código, donde se computen las métricas MAE, MSE, RMSE, MAPE y R2 aplicando los regresores OLS y KNN en DOS datasets de regresión (a elegir). Nota: Al menos uno de los datasets deberá ser externo a la librería de scikit-learn.

Se debe llevar a cabo:
- Exploración de los datos y descripción del dataset
- Tratamiento de outliers y valores perdidos (si los hay)
- Partición externa de datos
- Estandarización de los datos
- Selección de atributos mediante métodos supervisados (solo uno)
- Validación cruzada interna para optimizar los hiperparámetros de los algoritmos (si procede)
- Modelado
- Predicción y evaluación sobre el conjunto de test (resultados cuantitativos y cualitativos)
- Breve discusión comparando los resultados obtenidos con cada método.